Python script to retrieve the PnL of open margen possitions on Binance.
Right now the script only works for USDT pairs.

In [48]:
# https://python-binance.readthedocs.io/en/latest/margin.html#id12
try:
  from binance.client import Client
except Exception as e:
  !pip install python-binance
  from binance.client import Client

In [49]:
import time
import pandas as pd

In [50]:
key = 'YourApiKey'
sec = 'YourApiSecret'

In [51]:
client = Client(key, sec)

In [52]:
info = client.get_margin_account()

In [53]:
holdings = [i for i in info['userAssets'] if float(i['netAsset']) > 0]
# holdings

In [54]:
assets = [i['asset'] for i in holdings]
assets

['DOGE', 'DASH', 'LINK', 'SOL', 'BNB', 'ETH', 'ADA']

In [55]:
# The trades pairs
quote_currency = 'USDT'

In [56]:
# trades = client.get_margin_trades(symbol = f'ADA{quote_currency}') 
# trades
#  'isBuyer': True, = a buy
#  'isBuyer': False, = a sell

In [57]:
def get_entry_order(trades, current_amount):
  # Find the entry order so the first buy order for the current possition
  x = current_amount
  entry_order_id = None

  for i in trades[::-1]: # reverse the list
    if i['isBuyer']:
      x -= float(i['qty']) 
    elif i['isBuyer'] is False:
      x += float(i['qty'])
    if x <= 0.1:   
      entry_order_id = i['id']
      # print(f"Entry Order ID: {entry_order_id}")
      return entry_order_id

In [58]:
def slice_trades_to_entry_order(trades, entry_order_id):
  # Find the index of the entry order for current possition in the trades list
  for i, v in enumerate(trades):
      if int(v['id']) == int(entry_order_id):
        return trades[i:]

In [ ]:
my_finance = []
for asset in assets:
  print('Asset:', asset)

  trades = client.get_margin_trades(symbol = f'{asset}{quote_currency}') 
  total_Amount = float([i['netAsset'] for i in holdings if i['asset'] == asset][0])
  entry_order_id = get_entry_order(trades, total_Amount)
  if not entry_order_id:
    # print(trades)
    continue
  trades_ = slice_trades_to_entry_order(trades, entry_order_id)
  cost = 0

  for i in trades_:
    if i['isBuyer']:
      # total_Amount += float(i['qty'])
      cost += float(i['qty']) * float(i['price'])
    elif i['isBuyer'] is False:
      # total_Amount -= float(i['qty'])
      cost -= float(i['qty']) * float(i['price'])

  print(f'Asset: {asset}, Current Amount: {total_Amount}, Total Spend: {cost}, Average Entry Price: {cost / total_Amount}')
  # Average Cost per coin = Total spend ÷ total number of coins owned
  my_finance.append({'asset':asset, 'amount':total_Amount, 'cost':cost, 'avg_entry_price': cost / total_Amount})
  time.sleep(1)

print()
pd.DataFrame(my_finance)

In [60]:
prices = client.get_all_tickers()
# prices

In [ ]:
pnl_total = 0
pnl_ = []
for i in my_finance:
  price = [x for x in prices if x['symbol'] == i['asset']+quote_currency][0]['price']
  pnl = (float(price) - i['avg_entry_price']) /  i['avg_entry_price']
  pnl_total += pnl
  #pnl_.append({'asset':i['asset'], 'pnl %':pnl, 'pnl $': i['cost'] * pnl]})
  pnl_.append({
      'asset': i['asset'], 
      'pnl %': pnl, 
      'pnl $': i['cost'] * pnl,
      'invested': i['cost'],
      'avg_entry_price': i['avg_entry_price'],
      'last_price' : price
      })
  print(f'Asset:{i["asset"]}, PnL: {pnl}')

print()
print(f'Total PnL: {pnl_}')

In [ ]:
df = pd.DataFrame(pnl_).round(3)
df['pnl %'] = df['pnl %'] * 100
df.sort_values(by='pnl %',ascending=False, inplace=True)
df

In [ ]:
total_pnl = df['pnl $'].sum()
total_pnl